In [1]:
!pip install allennlp


  Using cached allennlp-2.10.1-py3-none-any.whl.metadata (21 kB)
INFO: pip is looking at multiple versions of allennlp to determine which version is compatible with other requirements. This could take a while.
  Using cached allennlp-2.10.0-py3-none-any.whl.metadata (20 kB)
  Using cached allennlp-2.9.3-py3-none-any.whl.metadata (19 kB)
  Using cached allennlp-2.9.2-py3-none-any.whl.metadata (19 kB)
  Using cached allennlp-2.9.1-py3-none-any.whl.metadata (19 kB)
  Using cached allennlp-2.9.0-py3-none-any.whl.metadata (18 kB)
  Using cached allennlp-2.8.0-py3-none-any.whl.metadata (17 kB)
  Using cached allennlp-2.7.0-py3-none-any.whl.metadata (17 kB)
INFO: pip is still looking at multiple versions of allennlp to determine which version is compatible with other requirements. This could take a while.
  Using cached allennlp-2.6.0-py3-none-any.whl.metadata (17 kB)
  Using cached allennlp-2.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached allennlp-2.4.0-py3-none-any.whl.metadata (17 kB

  error: subprocess-exited-with-error
  
  pip subprocess to install build dependencies did not run successfully.
  exit code: 1
  
  [326 lines of output]
    Using cached setuptools-72.1.0-py3-none-any.whl.metadata (6.6 kB)
    Using cached wheel-0.32.3-py2.py3-none-any.whl.metadata (2.1 kB)
    Using cached Cython-3.0.11-cp311-cp311-win_amd64.whl.metadata (3.2 kB)
    Using cached cymem-2.0.8-cp311-cp311-win_amd64.whl.metadata (8.6 kB)
    Using cached preshed-2.0.1.tar.gz (113 kB)
    Preparing metadata (setup.py): started
    Preparing metadata (setup.py): finished with status 'done'
    Using cached murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata (2.0 kB)
    Using cached thinc-7.0.8.tar.gz (1.9 MB)
    Preparing metadata (setup.py): started
    Preparing metadata (setup.py): finished with status 'done'
    Using cached blis-0.2.4.tar.gz (1.5 MB)
    Preparing metadata (setup.py): started
    Preparing metadata (setup.py): finished with status 'done'
    Using cached wasabi-

In [2]:
# Import necessary libraries
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from gensim.models import Word2Vec, FastText
from transformers import BertTokenizer, BertModel
import torch
from allennlp.modules.elmo import Elmo, batch_to_ids



ModuleNotFoundError: No module named 'allennlp'

In [ ]:
# Define your corpus
corpus = [
    'The cat sat on the mat',
    'The dog sat on the log',
    'Cats and dogs are great pets',
    'I love my cat and dog'
]



In [ ]:
# 1. One-Hot Encoding and Pad Sequences
print("One-Hot Encoding and Pad Sequences:")
vocab_size = 50
encoded_docs = [one_hot(doc, vocab_size) for doc in corpus]
max_length = 6
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)



In [ ]:
# Build a simple Embedding model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=8, input_length=max_length, name='embedding'))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()



In [ ]:
# 2. Word2Vec
print("\nWord2Vec Embeddings:")
sentences = [doc.split() for doc in corpus]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
vector_w2v = word2vec_model.wv['cat']
print(vector_w2v)



In [ ]:
# 3. GloVe
print("\nGloVe Embeddings:")
embeddings_index = {}
# Assume 'glove.6B.100d.txt' is downloaded and available
with open('glove.6B.100d.txt', 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coeffs
vector_glove = embeddings_index.get('cat')
print(vector_glove)




In [ ]:
# 4. FastText
print("\nFastText Embeddings:")
fasttext_model = FastText(sentences, vector_size=100, window=5, min_count=1, workers=4)
vector_ft = fasttext_model.wv['cat']
print(vector_ft)



In [ ]:
# 5. BERT
print("\nBERT Embeddings:")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')
inputs = tokenizer("The cat sat on the mat", return_tensors='pt')
with torch.no_grad():
    outputs = model_bert(**inputs)
    embeddings_bert = outputs.last_hidden_state
print(embeddings_bert)



In [ ]:
# 6. ELMo
print("\nELMo Embeddings:")
options_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_options.json"
weight_file = "https://allennlp.s3.amazonaws.com/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_weights.hdf5"
elmo = Elmo(options_file, weight_file, 1, dropout=0)
sentences_elmo = [['The', 'cat', 'sat', 'on', 'the', 'mat']]
character_ids = batch_to_ids(sentences_elmo)
embeddings_elmo = elmo(character_ids)['elmo_representations'][0]
print(embeddings_elmo)